# Introduction: Automated Machine Learning

In this notebook, we will see how to use [TPOT](https://epistasislab.github.io/tpot/api/), a Python library developed for automatic machine learning feature preprocessing, model selection, and hyperparameter tuning. Using [genetic programming](http://geneticprogramming.com/tutorial/), TPOT tries to find the best machine learning pipeline for a dataset by evaluating thousands of possibilites. 

The machine learning pipeline in this context consists of:

1. Feature Preprocessing
  * Imputing missing values and scaling values
  * Constructing new features such as polynomial transformations
2. Feature selection
  * Dimensionality reduction, for example using PCA and other techniques
3. Model Selection
  * Evaluting a number of machine learning models
4. Hyperparameter tuning
  * Finding the optimal settings of the model for the particular problem

TPOT is one of a class of methods known as [auto-ml (short for automated machine learning)](https://www.kdnuggets.com/2017/01/current-state-automated-machine-learning.html) which aim to simplify the work of the data scientist by automatically finding the optimal (or near-optimal) feature preprocessing steps and model for the problem. Machine learning is  typically a very time-consuming and knowledge-intensive part of a data science problem. Auto-ml is not designed to replace the data scientist, but rather free her to work on more important aspects of the complete problem, such as acquiring data and interpreting the model results. In effect, TPOT, and auto-ml in general, will in effect be a "data science assistant" that will be another tool among many used by data scientists. Machine learning is only one part of the data science process, and it still takes a human to weave the different aspects of a problem together into a complete working product.

Other entries in the field of auto - ml include:

* [Auto-sklearn](https://automl.github.io/auto-sklearn/stable/)
* [H20](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/welcome.html)
* [Google Cloud AutoML](https://cloud.google.com/automl/)


With that background, let's see how automated machine learning, the future of data science, works!

First, because we are working in Google Colab, we need to make sure to install `TPOT`. We can do that using a system command (which in Jupyter is proceeded by `!`).

In [1]:
# Install tpot on the server
!pip install tpot

In [2]:
# pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# Import the tpot regressor
from tpot import TPOTRegressor

# Problem Description

The task is a supervised regression problem: given [New York City energy data](http://www.nyc.gov/html/gbee/html/plan/ll84_scores.shtml), we want to build a model that can predict the Energy Star Score of a building. In a series of articles ([part one](https://towardsdatascience.com/a-complete-machine-learning-walk-through-in-python-part-one-c62152f39420), [part two](https://towardsdatascience.com/a-complete-machine-learning-project-walk-through-in-python-part-two-300f1f8147e2), [part three](https://towardsdatascience.com/a-complete-machine-learning-walk-through-in-python-part-three-388834e8804b), [code on GitHub](https://github.com/WillKoehrsen/machine-learning-project-walkthrough)), we built a complete machine learning solution for this problem. Using manual feature engineering, dimensionality reduction, model selection, and hyperparameter tuning, we were able to build a model that achieved a mean absolute error of 9.06 points (on a scale of 1-100) on the test set. 

## Dataset

The features contain a number of continuous numeric variables (such as energy use and area of the building) as well as two one-hot encoded categorical variables (borough and building type). There are a total of 82 features. 

All of the missing values have been encoded as `np.nan`, and TPOT will automatically perform missing value imputation. It also automatically scales the variables so we do not have to worry about normalizing the range of each feature. TPOT does both feature engineering and feature selection, so we will not transform any of the variables or remove extraneous features we think may be extraneous. 

We will read into the data from GitHub and take a brief look.


In [3]:
# Read in features from GitHub
train_features = pd.read_csv('https://raw.githubusercontent.com/WillKoehrsen/machine-learning-project-walkthrough/master/data/X_train.csv')
test_features = pd.read_csv('https://raw.githubusercontent.com/WillKoehrsen/machine-learning-project-walkthrough/master/data/X_test.csv')

# Read in labels from GitHub
train_labels = pd.read_csv('https://raw.githubusercontent.com/WillKoehrsen/machine-learning-project-walkthrough/master/data/Y_train.csv')
test_labels = pd.read_csv('https://raw.githubusercontent.com/WillKoehrsen/machine-learning-project-walkthrough/master/data/Y_test.csv')

print('Training features shape: ', train_features.shape)
print('Testing features shape:  ', test_features.shape)

Training features shape:  (6622, 82)
Testing features shape:   (2839, 82)


In [4]:
train_features.head()

,Order,Property Id,DOF Gross Floor Area,Largest Property Use Type - Gross Floor Area (ft²),Year Built,Number of Buildings - Self-reported,Occupancy,Site EUI (kBtu/ft²),Weather Normalized Site EUI (kBtu/ft²),Weather Normalized Site Electricity Intensity (kWh/ft²),...,Largest Property Use Type_Restaurant,Largest Property Use Type_Retail Store,Largest Property Use Type_Self-Storage Facility,Largest Property Use Type_Senior Care Community,Largest Property Use Type_Social/Meeting Hall,Largest Property Use Type_Strip Mall,Largest Property Use Type_Supermarket/Grocery Store,Largest Property Use Type_Urgent Care/Clinic/Other Outpatient,Largest Property Use Type_Wholesale Club/Supercenter,Largest Property Use Type_Worship Facility
0,13276,5849784,90300.0,77300.0,1950,1,100,126.0,136.8,5.2,...,0,0,0,0,0,0,0,0,0,0
1,7377,4398442,52000.0,52000.0,1926,1,100,95.4,102.0,4.7,...,0,0,0,0,0,0,0,0,0,0
2,9479,4665374,104700.0,105000.0,1954,1,100,40.4,40.0,3.8,...,0,0,0,0,0,0,0,0,0,0
3,14774,3393340,129333.0,129333.0,1992,1,100,157.1,163.1,16.9,...,0,0,0,1,0,0,0,0,0,0
4,3286,2704325,109896.0,116041.0,1927,1,100,62.3,68.2,3.5,...,0,0,0,0,0,0,0,0,0,0


In the code below, we convert to `numpy` arrays. This is not strictly necessary, but the labels should be converted to a one-dimensional vector (using `reshape` in the code below) or Scikit-Learn will show a warning message. 

In [5]:
# Convert to numpy arrays
training_features = np.array(train_features)
testing_features = np.array(test_features)

# Sklearn wants the labels as one-dimensional vectors
training_targets = np.array(train_labels).reshape((-1,))
testing_targets = np.array(test_labels).reshape((-1,))

After the minimal data preparation, we can create the TPOT optimizer. The syntax for [TPOT optimizers](https://epistasislab.github.io/tpot/using/#tpot-with-code) is designed to be as close to that for Scikit-Learn models as possible. 

The [default parameters for TPOT optimizers](https://epistasislab.github.io/tpot/api/) will test 100 populations of pipelines, each with 100 generations for a total of 10,000 pipelines. Using 10-fold cross validation, this represents 100,000 training runs. Even using Google Colab, this takes quite a while! To avoid running out of time on the Colab server (we get 12 hours of continuous run-time) we will set a maximum of 8 hours (480 minutes) for evaluation.  [TPOT is designed to be run for days](https://epistasislab.github.io/tpot/using/) to thoroughly evaluate many pipelines, but the results can be quite good even from a few hours of training. 

We set the following parameters in the call to the optimizer (feel free to change these and see how they affect the results):

* `scoring = neg_mean_absolute_error`: Our selected regression performance metric
* `max_time_mins = 480`: Limit evaluation to 8 hours
* `n_jobs = -1`: Use all available cores on the machine
* `verbosity = 2`: Show a limited amount of information while training
* `cv = 5`: Use 5-fold cross validation (default is 10)

After we create the optimizer, we `fit` it to the training data as with any Scikit-Learn machine learning model. This starts the optimization process which will continue for 8 hours. During training, we can see a limited amount of information (change the `verbosity` to see more or less).

In [6]:
# Create a tpot object with a few parameters
tpot = TPOTRegressor(scoring = 'neg_mean_absolute_error', 
                    max_time_mins = 10, 
                    n_jobs = -1,
                    verbosity = 2,
                    cv = 5)

In [7]:
# Fit the tpot model on the training data
tpot.fit(training_features, training_targets)

Imputing missing values in feature set


C:\Users\43587\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)



10.430438966666665 minutes have elapsed. TPOT will close down.
TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: ExtraTreesRegressor(input_matrix, bootstrap=True, max_features=0.9500000000000001, min_samples_leaf=1, min_samples_split=2, n_estimators=100)


TPOTRegressor(config_dict=None, crossover_rate=0.1, cv=5,
       disable_update_check=False, early_stop=None, generations=1000000,
       max_eval_time_mins=5, max_time_mins=10, memory=None,
       mutation_rate=0.9, n_jobs=-1, offspring_size=None,
       periodic_checkpoint_folder=None, population_size=100,
       random_state=None, scoring='neg_mean_absolute_error', subsample=1.0,
       use_dask=False, verbosity=2, warm_start=False)

Due to the time limit, we can see our model only was able to get through 15 generations. With 100 populations, this represents 1500 different individual pipelines that were evaluated, quit a few more than we would be able to try by hand! 

Once the model has finished training, we can see the optimal pipeline by printing the `fitted_pipeline`. This represents the complete pipeline with the best performance metric (in this case the highest `neg_mean_absolute_error`) from cross validation.

In [8]:
# Show the final model
print(tpot.fitted_pipeline_)

Pipeline(memory=None,
     steps=[('extratreesregressor', ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=None,
          max_features=0.9500000000000001, max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=2,
          min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
          oob_score=False, random_state=None, verbose=0, warm_start=False))])


The TPOT optimization process is stochastic, meaning that [each run will produce different results](https://epistasislab.github.io/tpot/using/). If you run this notebook again, don't worry if you see a different final pipeline!  

To save the pipeline for future use, we can export it to a Python script. 

In [9]:
# Export the pipeline as a python script file
tpot.export('tpot_exported_pipeline.py')

True

Since we are in a Google Colab notebook, this will save it on the server where our notebook is running and the file  will only persist while we are connected. To download the pipeline onto a local machine from Google's servers, we have to use the file helper functions (from `gooogle.colab`) to download it. 

The file can be [accessed on GitHub](https://github.com/WillKoehrsen/machine-learning-project-walkthrough/blob/master/auto_ml/tpot_exported_pipeline.py).

In [10]:
# Import file management
from google.colab import file

# Download the pipeline for local use
files.download('tpot_exported_pipeline.py')

ModuleNotFoundError: No module named 'google'

If we want to look at all of the evaluated pipelines, we can see the `.evaluated_individuals_` attribute of the fitted optimizer. Be careful about running this as it will print out all 1500 pipelines that were tested!

In [ ]:
# To examine all fitted models
# tpot.evaluated_individuals_

Finally, let's test the entire fitted pipeline on the test dataset. After evaluating all the pipelines, TPOT saves the best one and trains it on all the training data, so we can evaluate the best one using the optimizer `.score` method. This will display the negative mean squared error, our regression metric.

In [ ]:
# Evaluate the final model
print(tpot.score(testing_features, testing_targets))

In the series of articles where we solved this problem by hand, we built a `GradientBoostedRegressor` model that achieved 9.1 mean absolute error on the test set. Automated machine learning has significantly improved on that score with a drastic reduction in the amount of development time. This "data science assistant" feels like the future of data science! 

Here is the actual implementation of the final pipeline which I copy and pasted from the downloaded Python file. We can train and test it just to make sure that this score is correct! 

In [ ]:
# Imports that the final pipeline needs
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LassoLarsCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import Imputer
from tpot.builtins import StackingEstimator

# Preprocessing steps
imputer = Imputer(strategy="median")
imputer.fit(training_features)
training_features = imputer.transform(training_features)
testing_features = imputer.transform(testing_features)

# Final pipeline from TPOT
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=LassoLarsCV(normalize=True)),
    GradientBoostingRegressor(alpha=0.95, learning_rate=0.1, loss="lad", 
                              max_depth=7, max_features=0.75, 
                              min_samples_leaf=3, min_samples_split=18, 
                              n_estimators=100, subsample=0.60)
)

In [ ]:
# Fit on the training data
exported_pipeline.fit(training_features, training_targets)

After creating the optimized pipeline and training it, we can evaluate it on the testing set. As the models were not created with a `random_state`, we expect slightly different performance than the original results, but it should be fairly close.

In [ ]:
# Make predictions on the testing data
predictions = exported_pipeline.predict(testing_features)

print('Mean Absolute Error = %0.4f' % np.mean(abs(predictions - testing_targets)))

Sure enough, the mean absolute error is close to that from the optimizer `.score` method and considerably better than our manual pipeline building efforts. 

From here, we can use the optimization results and try to further fine-tune the pipeline, or we can move on to important phases of the data science workflow. If we use this as the final model, we could spend time trying to intrepret the model (perhaps using [LIME: Local Interpretable Model-Agnostic Explainations](https://www.oreilly.com/learning/introduction-to-local-interpretable-model-agnostic-explanations-lime)) or reporting our results. 

# Conclusions

We saw how to use TPOT, an auto-ml Python library for automatically designing a machine learning pipeline. This tool will evaluate thousands of feature construction, feature selection, model selection, and hyperparameter tuning configurations in order to simplify the job of the data scientist. As machine learning is just one part of the data science workflow, auto-ml will not replace the data scientist, but allow her to spend time on more important aspects of the process. Automated machine learning is still in its early stages, but it appears to be a promising method for optimizing the often tedious and frustrating task of finding the best machine learning pipeline. 

While being an early adopter does not always pay off, in this case, TPOT is mature enough to have a simple-to-use interface, but also new enough that you will be ahead of the curve if you are familiar with its use. With that in mind, find a problem and get out there are try to solve it! If you are looking for a place to start, [Kaggle](https://www.kaggle.com/) (The Self-Proclaimed Home of Data Science) has many datasets and problems that are well-suited for application of auto-ml. 